In [17]:
import numpy as np
import pandas as pd
import pyproj
import matplotlib.pyplot as plt
import rasterio as rio

In [18]:
def reproject(df, proj_crs):
    lon = df["lon"].to_numpy()
    lat = df["lat"].to_numpy()
    
    geo_crs = pyproj.CRS.from_proj4("+proj=longlat +a=6051800 +b=6051800 +no_defs ")
    # proj_crs = pyproj.CRS.from_proj4("+proj=eqc +lat_ts=0 +lat_0=0 +lon_0=0 +x_0=0 +y_0=0 +a=6051800 +b=6051800 +units=m +no_defs")
    # geo_crs = 'GEOGCS["Mars 2000",DATUM["D_Mars_2000",SPHEROID["Mars_2000_IAU_IAG",3396190.0,169.89444722361179]],PRIMEM["Greenwich",0],UNIT["Degree",0.017453292519943295]]'
    
    proj = pyproj.Transformer.from_crs(geo_crs, proj_crs)
        
    df["x"], df["y"] = proj.transform(lon, lat)
    
    dx = np.gradient(df["x"])
    dy = np.gradient(df["y"])

    theta = np.arctan(dy/dx)
    df["orbit_inc"] = np.rad2deg(theta)
    
    return df

In [19]:
df = pd.read_csv("/home/iganesh/arcdr_plains_test.csv")

In [20]:
# tiff_path = "/mnt/c/Users/iganesh/Work/Research/Venus/ImpactDeposits/Datasets/Magellan_mosaics/Venus_Magellan_LeftLook_mosaic_global_225m.tif"
tiff_path = "/home/iganesh/Venus_Magellan_LeftLook_mosaic_global_225m.tif"
tiff = rio.open(tiff_path, mode="r")
dfr = reproject(df, tiff.crs)
tiff.close()
print(dfr)

      FID  Field1   flag  flag_ambig  flag_amb_1  flag_bad  flag_badal  \
0  293100     430  32799         0.0         0.0       0.0         0.0   
1  293101     431  32799         0.0         0.0       0.0         0.0   
2  293102     432  32799         0.0         0.0       0.0         0.0   
3  293103     433  32799         0.0         0.0       0.0         0.0   
4  293104     434  32799         0.0         0.0       0.0         0.0   
5  293105     435  32799         0.0         0.0       0.0         0.0   
6  293106     436  32799         0.0         0.0       0.0         0.0   
7  293107     437  32799         0.0         0.0       0.0         0.0   
8  293108     438  32799         0.0         0.0       0.0         0.0   
9  293109     439  32799         0.0         0.0       0.0         0.0   

   flag_cbad  flag_cmark  flag_ephc  ...      pradius       ref  ref_corr  \
0        0.0         0.0        1.0  ...  6050.372070  0.085906  0.014721   
1        0.0         0.0       

In [23]:
def crop_footprints(df, raster):
    DN_list = []
    df["DN_mean"] = np.zeros((len(df))).astype(np.float32)
    df["DN_npix"] = np.zeros((len(df)))
    for i in range(300,len(df)):
        rec = df.iloc[i]
        x0 = rec["x"]
        y0 = rec["y"]
        a = rec["ysize"]*1000/2
        b = rec["xsize"]*1000/2
                
        ymin = y0 - 2*a
        ymax = y0 +2*a
        xmin = x0 - 2*a
        xmax = x0 + 2*a
                                
        A = np.deg2rad(90.0+rec["orbit_inc"])
        inds = rio.open(raster, mode="r")
        win=rio.windows.from_bounds(xmin, ymin, xmax, ymax, inds.transform)
        img = inds.read(1, window=win).astype(np.float32)
        inds.close()

        img[img>255] = np.nan
        img[img<0] = np.nan
        
        cols, rows = np.meshgrid(np.arange(win.col_off, win.col_off+win.width), np.arange(win.row_off, win.row_off+win.height))
        xs, ys = rio.transform.xy(inds.transform, rows, cols)
        ellipse = (((xs-x0)*np.cos(A) + (ys-y0)*np.sin(A))**2 / a**2) + (((xs-x0)*np.sin(A) - (ys-y0)*np.cos(A))**2 / b**2)

        plt.imshow(img, cmap="Greys")
        plt.colorbar()
        plt.contour(ellipse, levels=[1], colors="r")
        
        in_ellipse = img[ellipse<=1.0]
        mean = np.nanmean(in_ellipse)
        num_pix = np.count_nonzero(~np.isnan(img))
        # print(np.count_nonzero(ellipse<=1.0))
        rec["DN_mean"] = mean
        rec["DN_npix"] = num_pix
        
        break
        DN_list.append(in_ellipse)
        
    return df

In [24]:
crop_footprints(dfr, tiff_path)

,FID,Field1,flag,flag_ambig,flag_amb_1,flag_bad,flag_badal,flag_cbad,flag_cmark,flag_ephc,...,ref_corr,rms_slope,xsize,ysize,ref_fresn,x,y,orbit_inc,DN_mean,DN_npix
0,293100,430,32799,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.014721,2.260681,10.506870,18.333395,0.100627,-7.786085e+06,5.172416e+06,-79.501927,0.0,0.0
1,293101,431,32799,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.015177,2.459927,10.470333,18.307528,0.103154,-7.784158e+06,5.162015e+06,-79.510195,0.0,0.0
2,293102,432,32799,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.015185,2.551709,10.434047,18.281767,0.099605,-7.782240e+06,5.151649e+06,-79.552504,0.0,0.0
3,293103,433,32799,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.015921,2.091228,10.398007,18.256113,0.098411,-7.780342e+06,5.141320e+06,-79.595316,0.0,0.0
4,293104,434,32799,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.015718,2.178228,10.362213,18.230566,0.099938,-7.778453e+06,5.131025e+06,-79.621247,0.0,0.0
5,293105,435,32799,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.015052,2.133469,10.326660,18.205126,0.095759,-7.776578e+06,5.120766e+06,-79.673934,0.0,0.0
6,293106,436,32799,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.015230,2.198972,10.291348,18.179792,0.088636,-7.774721e+06,5.110541e+06,-79.709682,0.0,0.0
7,293107,437,32799,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.015684,2.240793,10.256273,18.154562,0.097560,-7.772871e+06,5.100352e+06,-79.745681,0.0,0.0
8,293108,438,32799,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.013615,3.056005,10.221434,18.129436,0.112485,-7.771041e+06,5.090196e+06,-79.791351,0.0,0.0
9,293109,439,32799,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.014746,2.757423,10.186828,18.104414,0.100447,-7.769220e+06,5.080074e+06,-79.801151,0.0,0.0


In [16]:
print(DN_list)

[]


In [15]:
for i in range(len(DN_list)):
    print(DN_list[i])